# Data analysis of Auction Model

In [32]:
import pandas as pd
import numpy as np

In [63]:
legacy_df = pd.read_csv("results_legacy_bidincrease.csv", index_col=0)

## Getting final row of each epoch

In [67]:
splits = np.where(legacy_df["Highest Bid"] == 0)[0]
splits = np.delete(splits, 0)
splits = np.append(splits, len(legacy_df["Highest Bid"]-1))
splits = splits - 1
print(splits)

[    100     201     302 ... 1009797 1009898 1009999]


In [72]:
final_prices_df = legacy_df.iloc[splits]
# Removing any rows which contain 0 or None
# These are edge cases where the model does not receive any bids in the first few timesteps
final_prices_df.drop(final_prices_df.index[np.where(final_prices_df["Highest Bid"] == 0)[0]], axis=0, inplace=True)

C:\Users\Will\AppData\Local\Temp\ipykernel_3384\3337094288.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prices_df.drop(final_prices_df.index[np.where(final_prices_df["Highest Bid"] == 0)[0]], axis=0, inplace=True)


## Calculating win ratio of EarlyBidder : SniperBidder

In [74]:
earlyBidderWinPercent = len(np.where(final_prices_df["Highest Bidder"] == "EarlyBidder")[0]) / len(final_prices_df["Highest Bidder"])
sniperBidderWinPercent = 1 - earlyBidderWinPercent

In [75]:
print(f'EarlyBidder win percentage: {earlyBidderWinPercent}')
print(f'SniperBidder win percentage: {sniperBidderWinPercent}')

EarlyBidder win percentage: 0.8346
SniperBidder win percentage: 0.1654
